In [24]:
# Import the necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import googlemaps

gmaps = googlemaps.Client(key='KEY')

# Make a GET request to the website
url = "https://www.rightmove.co.uk/property-to-rent/find.html?searchType=RENT&locationIdentifier=REGION%5E674&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=2&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&sortByPriceDescending=&_includeLetAgreed=on&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&letType=&letFurnishType=&houseFlatShare="
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the div elements with the class propertyCard-wrapper
properties = soup.find_all('div', {'class': 'propertyCard-wrapper'})

# Create an empty list to store the data
data = []

# Set the destination as Ilford station
destination = 'Ilford station, London, UK'

# Iterate over the properties and extract the relevant information
for property in properties:
    # Find the title, price, and URL
    title = property.find('h2', {'class': 'propertyCard-title'}).text
    price = property.find('span', {'class': 'propertyCard-priceValue'}).text
    link = property.find('a', {'class': 'propertyCard-link'})['href']
    link = "https://www.rightmove.co.uk"+link
    # Find the number of bedrooms
    details = property.find('div', {'class': 'propertyCard-details'}).text
    bedrooms = details.strip().split(" ")[0]

    # Check if the property is a penthouse
    penthouse = 'Penthouse' in title

    # Find the address
    address = property.find('address', {'class': 'propertyCard-address'}).text.replace("\n","").strip()

    # Use the API to calculate the distance from the property to Canary Wharf station
    distance = gmaps.distance_matrix([address], [destination])['rows'][0]['elements'][0]['distance']['text']

       # Store the data in a dictionary
    property_data = {
        'title': title,
        'price': price,
        'link': link,
        'bedrooms': bedrooms,
        'penthouse': penthouse,
        'address': address,
        'distance': distance
    }

    # Add the dictionary to the list
    data.append(property_data)

# Create a Pandas dataframe from the list of dictionaries
df = pd.DataFrame(data)
# Convert the distance column to a numeric data type
df['distance'] = pd.to_numeric(df['distance'].str.replace(' km',''))
# Convert the price column to a numeric data type
df['price'] = pd.to_numeric(df['price'].str.replace('£','').str.replace('pcm','').str.replace(',','').str.strip())
df['bedrooms'] = pd.to_numeric(df['bedrooms'])
# Divide the price by the number of bedrooms to get the price per bedroom
df['price_per_bedroom'] = df['price'] / df['bedrooms']
df['score'] = df['price_per_bedroom']*df['distance']
#df['yes/no'] = df.apply(lambda row: 0 if row['distance'] > 5 and row['price_per_bedroom'] > 500 else 1, axis=1)
# Print the dataframe
df


,title,price,link,bedrooms,penthouse,address,distance,price_per_bedroom,score
0,\n 2 bedroom maisonette,1550,https://www.rightmove.co.uk/properties/1300162...,2,False,"Brunswick Gardens, Ilford",6.5,775.000000,5037.500000
1,\n 3 bedroom flat,1800,https://www.rightmove.co.uk/properties/1302068...,3,False,"Cranbrook Road, IG6",3.7,600.000000,2220.000000
2,\n 4 bedroom end of terrace house ...,2300,https://www.rightmove.co.uk/properties/1295428...,4,False,"Knights Way, IG6",7.1,575.000000,4082.500000
3,\n 2 bedroom flat,1750,https://www.rightmove.co.uk/properties/1301714...,2,False,"High Road, Ilford, IG1",1.5,875.000000,1312.500000
4,\n 2 bedroom flat,1600,https://www.rightmove.co.uk/properties/1301707...,2,False,"Park Avenue , Ilford, IG1",1.2,800.000000,960.000000
5,\n 2 bedroom apartment,1550,https://www.rightmove.co.uk/properties/1301695...,2,False,"Memorial Heights, Newbury Park, IG2",3.6,775.000000,2790.000000
6,\n 4 bedroom semi-detached house ...,2500,https://www.rightmove.co.uk/properties/1301529...,4,False,"Werneth Hall Road, Ilford IG5",4.5,625.000000,2812.500000
7,\n 6 bedroom terraced house,3000,https://www.rightmove.co.uk/properties/1301310...,6,False,"Khartoum Road, IG1",1.6,500.000000,800.000000
8,\n 3 bedroom terraced house,2200,https://www.rightmove.co.uk/properties/1295397...,3,False,"Ilford, IG1",1.7,733.333333,1246.666667
9,\n 3 bedroom semi-detached house ...,2200,https://www.rightmove.co.uk/properties/1301237...,3,False,"Ilford, IG6",5.6,733.333333,4106.666667


In [28]:
df.to_csv('properties.csv', index=False)